# Tensorboard - get started

https://www.tensorflow.org/tensorboard/get_started

In [2]:
# Load the TensorBoard notebook extension
# -- not sure what this is, but it needs tensorflow installed
%load_ext tensorboard

In [3]:
import datetime
import os

In [4]:
import tensorflow as tf

## Globals

In [5]:
# Clear any logs from previous runs
%rm -rf ./logs/

## Data

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Create the Model

In [7]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

## Fit the model

In [8]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2211 - accuracy: 0.9348 - val_loss: 0.1051 - val_accuracy: 0.9682
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0970 - accuracy: 0.9708 - val_loss: 0.0787 - val_accuracy: 0.9744
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0694 - accuracy: 0.9775 - val_loss: 0.0755 - val_accuracy: 0.9776
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0536 - accuracy: 0.9830 - val_loss: 0.0638 - val_accuracy: 0.9805
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0428 - accuracy: 0.9863 - val_loss: 0.0670 - val_accuracy: 0.9803


## Tensorboard - Plug-in

In [9]:
%tensorboard --logdir logs/fit

## Using a Dataset

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

## loss & optimizer

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## stateful metrics (collections)

In [12]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

## Gradient Tape

In [13]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [14]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

## train

In [15]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24648185074329376, Accuracy: 92.74166870117188, Test Loss: 0.1205030083656311, Test Accuracy: 96.21000671386719
Epoch 2, Loss: 0.1059400886297226, Accuracy: 96.80332946777344, Test Loss: 0.08644083142280579, Test Accuracy: 97.27999877929688
Epoch 3, Loss: 0.07204297184944153, Accuracy: 97.8133316040039, Test Loss: 0.07168268412351608, Test Accuracy: 97.7699966430664
Epoch 4, Loss: 0.05589805915951729, Accuracy: 98.24666595458984, Test Loss: 0.06543651968240738, Test Accuracy: 97.80999755859375
Epoch 5, Loss: 0.04349242150783539, Accuracy: 98.57833099365234, Test Loss: 0.06226542592048645, Test Accuracy: 98.0199966430664


In [16]:
%tensorboard --logdir logs/gradient_tape